In [36]:
# Standard library imports
import datetime
import json
import os
import sys
import time
from concurrent.futures import ThreadPoolExecutor

# Third-party imports
from dotenv import load_dotenv
import duckdb
import pandas as pd
import requests
from web3 import Web3

# service_agreement_v1 contract (where publishing starts)
# 0x4B014C4B8DA1853243fBd9d404F10Db6Aa9FADFc
# 0xB20F6F3B9176D4B284bA26b80833ff5bFe6db28F -> ServiceAgreementV1
# 0xFfFFFFff00000000000000000000000000000001 -> TRAC

# Evironment variables
load_dotenv()
SUBSCAN_KEY = os.getenv("subscan_key")
MOTHERDUCK_KEY = os.getenv("motherDuck_token")
ONFINALITY_KEY = os.getenv("onfinality_key")
MAX_WORKERS = 2  # adjust this based on your system's capabilities


start_time = time.time()
# Connect to the Ethereum node using Websockets
w3 = Web3(Web3.HTTPProvider(f'https://origintrail.api.onfinality.io/rpc?apikey={ONFINALITY_KEY}'))

latest_block = w3.eth.block_number
last_blocks = (w3.eth.block_number) - 100

# Load ABI from json file
with open('data/ServiceAgreementV1.json', 'r') as file:
    serviceAgreementABI = json.load(file)

# Contract address and initialization
contract_address = '0xB20F6F3B9176D4B284bA26b80833ff5bFe6db28F'
contract = w3.eth.contract(address=contract_address, abi=serviceAgreementABI)

# Fetch past ServiceAgreementV1Created events
events_list = contract.events.ServiceAgreementV1Created.get_logs(fromBlock=3122337, toBlock=3122367)

if len(events_list) > 0:
    processed_events = [{
        'assetContract': item['args'].get('assetContract', ''),
        'startTime': item['args'].get('startTime', ''),
        'epochsNumber': item['args'].get('epochsNumber', ''),
        'epochLength': item['args'].get('epochLength', ''),
        'tokenAmount': item['args'].get('tokenAmount', ''),
        'event': item.get('event', ''),
        'tokenId': item['args'].get('tokenId', ''),
        'transactionHash': item.get('transactionHash', '').hex() if item.get('transactionHash') else '',
        'blockHash': item.get('blockHash', '').hex() if item.get('blockHash') else '',
        'blockNumber': item.get('blockNumber', ''),
        'address': item.get('address', '')
    } for item in events_list]
else:
    print("No events found for the specified blocks.")
    sys.exit()  # Exit the program


# Create DataFrame
df_assets = (pd.DataFrame(processed_events)
      .assign(tokenAmount=lambda x: x['tokenAmount'].astype(float) / 1e18,
              epochLength=lambda x: x['epochLength'].astype(float) / 86400,
              startTime=lambda x: pd.to_datetime(x['startTime'].apply(lambda y: datetime.datetime.utcfromtimestamp(y).isoformat())))
      .rename(columns={"assetContract":"ASSET_CONTRACT",
                       "startTime": "TIME_ASSET_CREATED",
                       "epochsNumber":"EPOCHS_NUMBER",
                       "epochLength":"EPOCH_LENGTH-(DAYS)",
                       "tokenAmount": "TRAC_PRICE",
                       "event":"EVENT",
                       "tokenId": "ASSET_ID",
                       "assetContract":"ASSET_CONTRACT",
                       "transactionHash":"TRANSACTION_HASH",
                       "blockHash":"BLOCK_HASH",
                       "blockNumber":"BLOCK_NUMBER",
                       "address":"EVENT_CONTRACT_ADDRESS"},
              errors="raise"))



# Get all transaction hashes
hashes = df_assets['TRANSACTION_HASH'].tolist()

def fetch_transaction_data(hash):
    subscan_url = "https://origintrail.api.subscan.io/api/scan/evm/transaction"
    headers = {
        "Content-Type": "application/json",
        "X-API-Key": SUBSCAN_KEY
    }
    data = {
        "hash": hash
    }
    response = requests.post(subscan_url, headers=headers, json=data).json()
    if response.get("code") == 0:
        data = response["data"]
        return {
            "message": response["message"],
            "generated_at": response["generated_at"],
            "hash": data["hash"],
            "from": data["from"],
            "to": data["to"]["address"]
        }

with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    hash_list = list(executor.map(fetch_transaction_data, hashes))

# Filter out any None values from the hash_list
hash_list = [h for h in hash_list if h is not None]

df_hash = ((pd.DataFrame(hash_list)
            .assign(generated_at=lambda x: pd.to_datetime(x['generated_at'].apply(lambda y: datetime.datetime.utcfromtimestamp(y).isoformat()))))
            .rename(columns={"message":"MESSAGE",
                            "generated_at":"TIME_OF_TRANSACTION",
                            "hash":"TRANSACTION_HASH",
                            "from":"PUBLISHER_ADDRESS",
                            "to":"SENT_ADDRESS"},
                    errors="raise"))

df = pd.merge(df_assets, df_hash, on='TRANSACTION_HASH', how='left')

# Filter rows based on the MESSAGE column
df = df[df['MESSAGE'] == 'Success']

df = df[['MESSAGE',
         'ASSET_ID',
         'BLOCK_NUMBER',
         'TIME_ASSET_CREATED',
         'TIME_OF_TRANSACTION',
         'TRAC_PRICE',
         'EPOCHS_NUMBER',
         'EPOCH_LENGTH-(DAYS)',
         'PUBLISHER_ADDRESS',
         'SENT_ADDRESS',
         'TRANSACTION_HASH',
         'BLOCK_HASH']]



In [37]:
df

,MESSAGE,ASSET_ID,BLOCK_NUMBER,TIME_ASSET_CREATED,TIME_OF_TRANSACTION,TRAC_PRICE,EPOCHS_NUMBER,EPOCH_LENGTH-(DAYS),PUBLISHER_ADDRESS,SENT_ADDRESS,TRANSACTION_HASH,BLOCK_HASH
0,Success,563659,3122337,2023-08-26 04:07:24,2023-08-26 05:48:19,0.021240,5,90.0,0x19f5e80f3985c11195a1ee4ac012dadef19bbe90,0xb20016d23c77612334bd2885636980937da60b59,0x59c3adf8487d5d0dee159eec1e3cc41e90a771dd8c6d...,0xffaed74521c07a2e2bc52d0018640d20428f0161f802...
1,Success,563660,3122337,2023-08-26 04:07:24,2023-08-26 05:48:19,0.021240,5,90.0,0xa0a6ff7ee2d71d04c20195b8a919fafff513a9d4,0xb20016d23c77612334bd2885636980937da60b59,0x2be8b4469bfc8f825cfc632a2215e596247789460beb...,0xffaed74521c07a2e2bc52d0018640d20428f0161f802...
2,Success,563661,3122337,2023-08-26 04:07:24,2023-08-26 05:48:19,0.022357,5,90.0,0xdc462afc421ae7730cea2884dffefa1581bf0b67,0xb20016d23c77612334bd2885636980937da60b59,0x777d31d3aa706aa630b32a78d60c25a0584feaa48d08...,0xffaed74521c07a2e2bc52d0018640d20428f0161f802...
3,Success,563662,3122337,2023-08-26 04:07:24,2023-08-26 05:48:19,0.021240,5,90.0,0xb6d8ccf61cd965946248085d365fe670f2b5f563,0xb20016d23c77612334bd2885636980937da60b59,0x6a34a3893036e5816cdd196bc7f37e4516a6f324bb8a...,0xffaed74521c07a2e2bc52d0018640d20428f0161f802...
4,Success,563663,3122339,2023-08-26 04:07:48,2023-08-26 05:48:19,0.630469,5,90.0,0x13891d2f0120dfd4c7bb8426b496c68353d06cda,0xb20016d23c77612334bd2885636980937da60b59,0x4e6129f15e0c50d08c90f7d12a4b86c277fc56d0bf6f...,0xaed7a39162703ecd995515a2036f602fe9c4cededdbe...
5,Success,563664,3122339,2023-08-26 04:07:48,2023-08-26 05:48:20,0.021240,5,90.0,0x60f763c37981fda8cb96644e607c9611ea6133a0,0xb20016d23c77612334bd2885636980937da60b59,0xf79d8f3473ae4d62e10435040c0b8e8e2220f1afa9db...,0xaed7a39162703ecd995515a2036f602fe9c4cededdbe...
6,Success,563665,3122341,2023-08-26 04:08:12,2023-08-26 05:48:20,0.021240,5,90.0,0xb6d8ccf61cd965946248085d365fe670f2b5f563,0xb20016d23c77612334bd2885636980937da60b59,0xe1394a22e8c10817839d93a68127bb74c5b4e2be6aad...,0xe37795f354071c6f78b33eebfb5e0b4fb5fdd0011a84...
7,Success,563666,3122342,2023-08-26 04:08:24,2023-08-26 05:48:20,0.079834,5,90.0,0xdc462afc421ae7730cea2884dffefa1581bf0b67,0xb20016d23c77612334bd2885636980937da60b59,0x5c8d9c1507800e574b2c95248c49aace3f43958bd90d...,0x1488091491b08e71a2aede56ee82b66b86f891bca896...
8,Success,563667,3122343,2023-08-26 04:08:36,2023-08-26 05:48:20,0.491602,5,90.0,0x13891d2f0120dfd4c7bb8426b496c68353d06cda,0xb20016d23c77612334bd2885636980937da60b59,0x5e8681d42fc393b3db67b55f51ab6e8ba6b720ef482b...,0x732f869b859092d43b40516d950b983efb7d010caf8e...
9,Success,563668,3122343,2023-08-26 04:08:36,2023-08-26 05:48:20,0.021240,5,90.0,0xec247e837b3323afc350271cdf547acd5d8f2043,0xb20016d23c77612334bd2885636980937da60b59,0x7a0dffe5652e6990671f72d8e4a0ad7144915758a856...,0x732f869b859092d43b40516d950b983efb7d010caf8e...


In [38]:
df_hash

,MESSAGE,TIME_OF_TRANSACTION,TRANSACTION_HASH,PUBLISHER_ADDRESS,SENT_ADDRESS
0,Success,2023-08-26 05:48:19,0x59c3adf8487d5d0dee159eec1e3cc41e90a771dd8c6d...,0x19f5e80f3985c11195a1ee4ac012dadef19bbe90,0xb20016d23c77612334bd2885636980937da60b59
1,Success,2023-08-26 05:48:19,0x2be8b4469bfc8f825cfc632a2215e596247789460beb...,0xa0a6ff7ee2d71d04c20195b8a919fafff513a9d4,0xb20016d23c77612334bd2885636980937da60b59
2,Success,2023-08-26 05:48:19,0x777d31d3aa706aa630b32a78d60c25a0584feaa48d08...,0xdc462afc421ae7730cea2884dffefa1581bf0b67,0xb20016d23c77612334bd2885636980937da60b59
3,Success,2023-08-26 05:48:19,0x6a34a3893036e5816cdd196bc7f37e4516a6f324bb8a...,0xb6d8ccf61cd965946248085d365fe670f2b5f563,0xb20016d23c77612334bd2885636980937da60b59
4,Success,2023-08-26 05:48:19,0x4e6129f15e0c50d08c90f7d12a4b86c277fc56d0bf6f...,0x13891d2f0120dfd4c7bb8426b496c68353d06cda,0xb20016d23c77612334bd2885636980937da60b59
5,Success,2023-08-26 05:48:20,0xf79d8f3473ae4d62e10435040c0b8e8e2220f1afa9db...,0x60f763c37981fda8cb96644e607c9611ea6133a0,0xb20016d23c77612334bd2885636980937da60b59
6,Success,2023-08-26 05:48:20,0xe1394a22e8c10817839d93a68127bb74c5b4e2be6aad...,0xb6d8ccf61cd965946248085d365fe670f2b5f563,0xb20016d23c77612334bd2885636980937da60b59
7,Success,2023-08-26 05:48:20,0x5c8d9c1507800e574b2c95248c49aace3f43958bd90d...,0xdc462afc421ae7730cea2884dffefa1581bf0b67,0xb20016d23c77612334bd2885636980937da60b59
8,Success,2023-08-26 05:48:20,0x5e8681d42fc393b3db67b55f51ab6e8ba6b720ef482b...,0x13891d2f0120dfd4c7bb8426b496c68353d06cda,0xb20016d23c77612334bd2885636980937da60b59
9,Success,2023-08-26 05:48:20,0x7a0dffe5652e6990671f72d8e4a0ad7144915758a856...,0xec247e837b3323afc350271cdf547acd5d8f2043,0xb20016d23c77612334bd2885636980937da60b59


In [39]:
df_assets

,ASSET_CONTRACT,TIME_ASSET_CREATED,EPOCHS_NUMBER,EPOCH_LENGTH-(DAYS),TRAC_PRICE,EVENT,ASSET_ID,TRANSACTION_HASH,BLOCK_HASH,BLOCK_NUMBER,EVENT_CONTRACT_ADDRESS
0,0x5cAC41237127F94c2D21dAe0b14bFeFa99880630,2023-08-26 04:07:24,5,90.0,0.021240,ServiceAgreementV1Created,563659,0x59c3adf8487d5d0dee159eec1e3cc41e90a771dd8c6d...,0xffaed74521c07a2e2bc52d0018640d20428f0161f802...,3122337,0xB20F6F3B9176D4B284bA26b80833ff5bFe6db28F
1,0x5cAC41237127F94c2D21dAe0b14bFeFa99880630,2023-08-26 04:07:24,5,90.0,0.021240,ServiceAgreementV1Created,563660,0x2be8b4469bfc8f825cfc632a2215e596247789460beb...,0xffaed74521c07a2e2bc52d0018640d20428f0161f802...,3122337,0xB20F6F3B9176D4B284bA26b80833ff5bFe6db28F
2,0x5cAC41237127F94c2D21dAe0b14bFeFa99880630,2023-08-26 04:07:24,5,90.0,0.022357,ServiceAgreementV1Created,563661,0x777d31d3aa706aa630b32a78d60c25a0584feaa48d08...,0xffaed74521c07a2e2bc52d0018640d20428f0161f802...,3122337,0xB20F6F3B9176D4B284bA26b80833ff5bFe6db28F
3,0x5cAC41237127F94c2D21dAe0b14bFeFa99880630,2023-08-26 04:07:24,5,90.0,0.021240,ServiceAgreementV1Created,563662,0x6a34a3893036e5816cdd196bc7f37e4516a6f324bb8a...,0xffaed74521c07a2e2bc52d0018640d20428f0161f802...,3122337,0xB20F6F3B9176D4B284bA26b80833ff5bFe6db28F
4,0x5cAC41237127F94c2D21dAe0b14bFeFa99880630,2023-08-26 04:07:48,5,90.0,0.630469,ServiceAgreementV1Created,563663,0x4e6129f15e0c50d08c90f7d12a4b86c277fc56d0bf6f...,0xaed7a39162703ecd995515a2036f602fe9c4cededdbe...,3122339,0xB20F6F3B9176D4B284bA26b80833ff5bFe6db28F
5,0x5cAC41237127F94c2D21dAe0b14bFeFa99880630,2023-08-26 04:07:48,5,90.0,0.021240,ServiceAgreementV1Created,563664,0xf79d8f3473ae4d62e10435040c0b8e8e2220f1afa9db...,0xaed7a39162703ecd995515a2036f602fe9c4cededdbe...,3122339,0xB20F6F3B9176D4B284bA26b80833ff5bFe6db28F
6,0x5cAC41237127F94c2D21dAe0b14bFeFa99880630,2023-08-26 04:08:12,5,90.0,0.021240,ServiceAgreementV1Created,563665,0xe1394a22e8c10817839d93a68127bb74c5b4e2be6aad...,0xe37795f354071c6f78b33eebfb5e0b4fb5fdd0011a84...,3122341,0xB20F6F3B9176D4B284bA26b80833ff5bFe6db28F
7,0x5cAC41237127F94c2D21dAe0b14bFeFa99880630,2023-08-26 04:08:24,5,90.0,0.079834,ServiceAgreementV1Created,563666,0x5c8d9c1507800e574b2c95248c49aace3f43958bd90d...,0x1488091491b08e71a2aede56ee82b66b86f891bca896...,3122342,0xB20F6F3B9176D4B284bA26b80833ff5bFe6db28F
8,0x5cAC41237127F94c2D21dAe0b14bFeFa99880630,2023-08-26 04:08:36,5,90.0,0.491602,ServiceAgreementV1Created,563667,0x5e8681d42fc393b3db67b55f51ab6e8ba6b720ef482b...,0x732f869b859092d43b40516d950b983efb7d010caf8e...,3122343,0xB20F6F3B9176D4B284bA26b80833ff5bFe6db28F
9,0x5cAC41237127F94c2D21dAe0b14bFeFa99880630,2023-08-26 04:08:36,5,90.0,0.021240,ServiceAgreementV1Created,563668,0x7a0dffe5652e6990671f72d8e4a0ad7144915758a856...,0x732f869b859092d43b40516d950b983efb7d010caf8e...,3122343,0xB20F6F3B9176D4B284bA26b80833ff5bFe6db28F


In [18]:
last_blocks = 3122337
latest_block = last_blocks + 10


processed_events = []
for block in range(last_blocks, latest_block+1):

    # Fetch past ServiceAgreementV1Created events
    events_for_block = contract.events.ServiceAgreementV1Created.get_logs(fromBlock=block, toBlock=block)

    if len(events_for_block) > 0:
        processed_events.extend([{
            'assetContract': item['args'].get('assetContract', ''),
            'startTime': item['args'].get('startTime', ''),
            'epochsNumber': item['args'].get('epochsNumber', ''),
            'epochLength': item['args'].get('epochLength', ''),
            'tokenAmount': item['args'].get('tokenAmount', ''),
            'event': item.get('event', ''),
            'tokenId': item['args'].get('tokenId', ''),
            'transactionHash': item.get('transactionHash', '').hex() if item.get('transactionHash') else '',
            'blockHash': item.get('blockHash', '').hex() if item.get('blockHash') else '',
            'blockNumber': item.get('blockNumber', ''),
            'address': item.get('address', '')
        } for item in events_for_block])
        
df2 = pd.DataFrame(processed_events)

# Create DataFrame
df_assets = (pd.DataFrame(processed_events)
      .assign(tokenAmount=lambda x: x['tokenAmount'].astype(float) / 1e18,
              epochLength=lambda x: x['epochLength'].astype(float) / 86400,
              startTime=lambda x: pd.to_datetime(x['startTime'].apply(lambda y: datetime.datetime.utcfromtimestamp(y).isoformat())))
      .rename(columns={"assetContract":"ASSET_CONTRACT",
                       "startTime": "TIME_ASSET_CREATED",
                       "epochsNumber":"EPOCHS_NUMBER",
                       "epochLength":"EPOCH_LENGTH-(DAYS)",
                       "tokenAmount": "TRAC_PRICE",
                       "event":"EVENT",
                       "tokenId": "ASSET_ID",
                       "assetContract":"ASSET_CONTRACT",
                       "transactionHash":"TRANSACTION_HASH",
                       "blockHash":"BLOCK_HASH",
                       "blockNumber":"BLOCK_NUMBER",
                       "address":"EVENT_CONTRACT_ADDRESS"},
              errors="raise"))



# Get all transaction hashes
hashes = df_assets['TRANSACTION_HASH'].tolist()

df_assets

,ASSET_CONTRACT,TIME_ASSET_CREATED,EPOCHS_NUMBER,EPOCH_LENGTH-(DAYS),TRAC_PRICE,EVENT,ASSET_ID,TRANSACTION_HASH,BLOCK_HASH,BLOCK_NUMBER,EVENT_CONTRACT_ADDRESS
0,0x5cAC41237127F94c2D21dAe0b14bFeFa99880630,2023-08-26 04:07:24,5,90.0,0.021240,ServiceAgreementV1Created,563659,0x59c3adf8487d5d0dee159eec1e3cc41e90a771dd8c6d...,0xffaed74521c07a2e2bc52d0018640d20428f0161f802...,3122337,0xB20F6F3B9176D4B284bA26b80833ff5bFe6db28F
1,0x5cAC41237127F94c2D21dAe0b14bFeFa99880630,2023-08-26 04:07:24,5,90.0,0.021240,ServiceAgreementV1Created,563660,0x2be8b4469bfc8f825cfc632a2215e596247789460beb...,0xffaed74521c07a2e2bc52d0018640d20428f0161f802...,3122337,0xB20F6F3B9176D4B284bA26b80833ff5bFe6db28F
2,0x5cAC41237127F94c2D21dAe0b14bFeFa99880630,2023-08-26 04:07:24,5,90.0,0.022357,ServiceAgreementV1Created,563661,0x777d31d3aa706aa630b32a78d60c25a0584feaa48d08...,0xffaed74521c07a2e2bc52d0018640d20428f0161f802...,3122337,0xB20F6F3B9176D4B284bA26b80833ff5bFe6db28F
3,0x5cAC41237127F94c2D21dAe0b14bFeFa99880630,2023-08-26 04:07:24,5,90.0,0.021240,ServiceAgreementV1Created,563662,0x6a34a3893036e5816cdd196bc7f37e4516a6f324bb8a...,0xffaed74521c07a2e2bc52d0018640d20428f0161f802...,3122337,0xB20F6F3B9176D4B284bA26b80833ff5bFe6db28F
4,0x5cAC41237127F94c2D21dAe0b14bFeFa99880630,2023-08-26 04:07:48,5,90.0,0.630469,ServiceAgreementV1Created,563663,0x4e6129f15e0c50d08c90f7d12a4b86c277fc56d0bf6f...,0xaed7a39162703ecd995515a2036f602fe9c4cededdbe...,3122339,0xB20F6F3B9176D4B284bA26b80833ff5bFe6db28F
5,0x5cAC41237127F94c2D21dAe0b14bFeFa99880630,2023-08-26 04:07:48,5,90.0,0.021240,ServiceAgreementV1Created,563664,0xf79d8f3473ae4d62e10435040c0b8e8e2220f1afa9db...,0xaed7a39162703ecd995515a2036f602fe9c4cededdbe...,3122339,0xB20F6F3B9176D4B284bA26b80833ff5bFe6db28F
6,0x5cAC41237127F94c2D21dAe0b14bFeFa99880630,2023-08-26 04:08:12,5,90.0,0.021240,ServiceAgreementV1Created,563665,0xe1394a22e8c10817839d93a68127bb74c5b4e2be6aad...,0xe37795f354071c6f78b33eebfb5e0b4fb5fdd0011a84...,3122341,0xB20F6F3B9176D4B284bA26b80833ff5bFe6db28F
7,0x5cAC41237127F94c2D21dAe0b14bFeFa99880630,2023-08-26 04:08:24,5,90.0,0.079834,ServiceAgreementV1Created,563666,0x5c8d9c1507800e574b2c95248c49aace3f43958bd90d...,0x1488091491b08e71a2aede56ee82b66b86f891bca896...,3122342,0xB20F6F3B9176D4B284bA26b80833ff5bFe6db28F
8,0x5cAC41237127F94c2D21dAe0b14bFeFa99880630,2023-08-26 04:08:36,5,90.0,0.491602,ServiceAgreementV1Created,563667,0x5e8681d42fc393b3db67b55f51ab6e8ba6b720ef482b...,0x732f869b859092d43b40516d950b983efb7d010caf8e...,3122343,0xB20F6F3B9176D4B284bA26b80833ff5bFe6db28F
9,0x5cAC41237127F94c2D21dAe0b14bFeFa99880630,2023-08-26 04:08:36,5,90.0,0.021240,ServiceAgreementV1Created,563668,0x7a0dffe5652e6990671f72d8e4a0ad7144915758a856...,0x732f869b859092d43b40516d950b983efb7d010caf8e...,3122343,0xB20F6F3B9176D4B284bA26b80833ff5bFe6db28F


In [23]:


def fetch_transaction_data(hash):
    subscan_url = "https://origintrail.api.subscan.io/api/scan/evm/transaction"
    headers = {
        "Content-Type": "application/json",
        "X-API-Key": SUBSCAN_KEY
    }
    data = {
        "hash": hash
    }
    response = requests.post(subscan_url, headers=headers, json=data).json()
    if response.get("code") == 0:
        data = response["data"]
        return {
            "message": response["message"],
            "generated_at": response["generated_at"],
            "hash": data["hash"],
            "from": data["from"],
            "to": data["to"]["address"]
        }

with ThreadPoolExecutor(max_workers=2) as executor:
    hash_list = list(executor.map(fetch_transaction_data, hashes))

# Filter out any None values from the hash_list
hash_list = [h for h in hash_list if h is not None]

df_hash = ((pd.DataFrame(hash_list)
            .assign(generated_at=lambda x: pd.to_datetime(x['generated_at'].apply(lambda y: datetime.datetime.utcfromtimestamp(y).isoformat()))))
            .rename(columns={"message":"MESSAGE",
                            "generated_at":"TIME_OF_TRANSACTION",
                            "hash":"TRANSACTION_HASH",
                            "from":"PUBLISHER_ADDRESS",
                            "to":"SENT_ADDRESS"},
                    errors="raise"))

df_hash

,MESSAGE,TIME_OF_TRANSACTION,TRANSACTION_HASH,PUBLISHER_ADDRESS,SENT_ADDRESS
0,Success,2023-08-26 05:38:49,0x59c3adf8487d5d0dee159eec1e3cc41e90a771dd8c6d...,0x19f5e80f3985c11195a1ee4ac012dadef19bbe90,0xb20016d23c77612334bd2885636980937da60b59
1,Success,2023-08-26 05:38:49,0x2be8b4469bfc8f825cfc632a2215e596247789460beb...,0xa0a6ff7ee2d71d04c20195b8a919fafff513a9d4,0xb20016d23c77612334bd2885636980937da60b59
2,Success,2023-08-26 05:38:49,0x777d31d3aa706aa630b32a78d60c25a0584feaa48d08...,0xdc462afc421ae7730cea2884dffefa1581bf0b67,0xb20016d23c77612334bd2885636980937da60b59
3,Success,2023-08-26 05:38:49,0x6a34a3893036e5816cdd196bc7f37e4516a6f324bb8a...,0xb6d8ccf61cd965946248085d365fe670f2b5f563,0xb20016d23c77612334bd2885636980937da60b59
4,Success,2023-08-26 05:38:50,0x4e6129f15e0c50d08c90f7d12a4b86c277fc56d0bf6f...,0x13891d2f0120dfd4c7bb8426b496c68353d06cda,0xb20016d23c77612334bd2885636980937da60b59
5,Success,2023-08-26 05:38:49,0xf79d8f3473ae4d62e10435040c0b8e8e2220f1afa9db...,0x60f763c37981fda8cb96644e607c9611ea6133a0,0xb20016d23c77612334bd2885636980937da60b59
6,Success,2023-08-26 05:38:50,0xe1394a22e8c10817839d93a68127bb74c5b4e2be6aad...,0xb6d8ccf61cd965946248085d365fe670f2b5f563,0xb20016d23c77612334bd2885636980937da60b59
7,Success,2023-08-26 05:38:50,0x5c8d9c1507800e574b2c95248c49aace3f43958bd90d...,0xdc462afc421ae7730cea2884dffefa1581bf0b67,0xb20016d23c77612334bd2885636980937da60b59
8,Success,2023-08-26 05:38:50,0x5e8681d42fc393b3db67b55f51ab6e8ba6b720ef482b...,0x13891d2f0120dfd4c7bb8426b496c68353d06cda,0xb20016d23c77612334bd2885636980937da60b59
9,Success,2023-08-26 05:38:50,0x7a0dffe5652e6990671f72d8e4a0ad7144915758a856...,0xec247e837b3323afc350271cdf547acd5d8f2043,0xb20016d23c77612334bd2885636980937da60b59


In [17]:
hash="0x5e8681d42fc393b3db67b55f51ab6e8ba6b720ef482b77c78f1ae973807a623b"

subscan_url = "https://origintrail.api.subscan.io/api/scan/evm/transaction"
headers = {
    "Content-Type": "application/json",
    "X-API-Key": SUBSCAN_KEY
}
data = {
    "hash": hash
}
response = requests.post(subscan_url, headers=headers, json=data).json()

response

{'code': 0,
 'message': 'Success',
 'generated_at': 1693028149,
 'data': {'block_num': 3122343,
  'hash': '0x5e8681d42fc393b3db67b55f51ab6e8ba6b720ef482b77c78f1ae973807a623b',
  'block_timestamp': 1693022916,
  'success': True,
  'error_type': '',
  'error_msg': 'Returned',
  'trace_error_msg': '',
  'from': '0x13891d2f0120dfd4c7bb8426b496c68353d06cda',
  'to': {'address': '0xb20016d23c77612334bd2885636980937da60b59',
   'is_contract': True},
  'contract': '',
  'value': '0',
  'gas_limit': '450000',
  'gas_price': '8',
  'gas_used': '397038',
  'nonce': 70751,
  'input_data': '0x22729ad037434ebceb46af978031b913e3d8b3b4bb4123a5a96d0d9b687155605c7122240000000000000000000000000000000000000000000000000000000000000d1c00000000000000000000000000000000000000000000000000000000000000170000000000000000000000000000000000000000000000000000000000000017000000000000000000000000000000000000000000000000000000000000000500000000000000000000000000000000000000000000000006d2850437d6e900000000000000000000000